# Support Vector Machine and Random Trees (Random Forest) Classification

This notebook is to be used with the ArcGIS Pro Python API to classify SENTINEL-2 images into self defined land cover classes using SVM and RF classifiers.

Author: Adian Dawuda | adian.dawuda@stud.plus.ac.at

In [2]:
# Import system modules
import arcpy
from arcpy.ia import *

In [5]:
# Clip Sentinel 2 raster to San Marino Boundaries
arcpy.management.Clip(
    "SanMarinoS2_full.tif", 
    "-20037507.0671618 -30240971.9583861 20037507.0671618 18460513.2470149", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\AnalysisAndModelingFinalVisualInt.gdb\SanMarinoS2_all_bands", 
    "World_Countries", 
    "1.79e+308", 
    "ClippingGeometry", 
    "NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Users\\Adian\\Documents\\ArcGIS\\Projects\\AnalysisAndModelingFinalVisualInt\\AnalysisAndModelingFinalVisualInt.gdb\\SanMarinoS2_all_bands'>

In [6]:
# Create a copy of the image with bands 2, 3, 4, and 8
arcpy.management.MakeRasterLayer(
    "SanMarinoS2_all_bands", 
    "SanMarinoS2", 
    '', 
    '1378670 5448230 1394740 5465090 PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]', 
    "2;3;4;8"
)

<Result 'SanMarinoS2'>

# SVM

In [7]:
# Train SVM and Generate an Esri classifier definition file (.ecd)
arcpy.ia.TrainSupportVectorMachineClassifier(
    "SanMarinoS2.tif", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\samples.shp", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\SanMarinoTrainedSVM.ecd", 
    '', 
    500, 
    "COLOR;MEAN;STD;COUNT;COMPACTNESS;RECTANGULARITY", 
    None
)

<geoprocessing server result object object at 0x00000255334C1B10>

In [8]:
# Classify the Sentinel image using the Esri classifier definition file (.ecd)
out_raster_dataset = arcpy.ia.ClassifyRaster(
    "SanMarinoS2.tif",
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\SanMarinoTrainedSVM.ecd",
    ''
);
out_raster_dataset.save(
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\LandCoverSanMarinoSVM.crf"
)

RuntimeError: Ungültiger Zeiger 

# RF

In [9]:
# Train RF and Classify raster
arcpy.stats.Forest(
    "PREDICT_RASTER",
    "samples", 
    "Classcode", 
    "CATEGORICAL",
    None, 
    None,
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2 #", 
    None, 
    None, 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\rf10.crf", 
    None, 
    None, 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2 C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\TrainedFeatures10.shp", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\VariableImportance10.dbf", 
    "TRUE", 
    100,
    None,
    None,
    100, 
    None, 
    10, 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\classificationPerformanceTable10.dbf", 
    None, 
    "TRUE",
    1,
    "FALSE"
)

<geoprocessing server result object object at 0x0000025526D1FED0>

## Accuracy Assessment

### SVM

In [8]:
# Create points for SVM
arcpy.ia.CreateAccuracyAssessmentPoints(
    "LandCoverSanMarinoSVM", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\AccuracyAssessmentPointsSVM.shp", 
    "CLASSIFIED", 
    50, 
    "STRATIFIED_RANDOM",
    None
)

<geoprocessing server result object object at 0x0000015B4D055CF0>

Visual interpretation is now necessary to fill in the 'Ground Truth' field

In [11]:
# Calculate confusion matrix
arcpy.ia.ComputeConfusionMatrix(
    "AccuracyAssessmentPointsSVM", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\AnalysisAndModelingFinalVisualInt.gdb\AccuracySVM"
)

<geoprocessing server result object object at 0x00000254E7DF2780>

### RF

In [12]:
# Create points for RF
# 8 or 16 bpp raster must be used as input
arcpy.ia.CreateAccuracyAssessmentPoints(
    "rf10_16bit.crf", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\AccuracyAssessmentPointsRF_10.shp", 
    "CLASSIFIED", 
    50, 
    "STRATIFIED_RANDOM",
    None
)

<geoprocessing server result object object at 0x0000015B4D0557E0>

In [1]:
# Update ground truth identifiers with class values

# Path to the workspace and the feature class
workspace = r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt"

feature_class = "AccuracyAssessmentPointsRF_10"

# Field name to be updated
field_name = "Classified"


# Map values to match simple land cover
value_mapping = {
    0: 10,
    1: 20,
    2: 30
}

try:
    arcpy.env.workspace = workspace

    # Start an edit session
    edit = arcpy.da.Editor(workspace)
    edit.startEditing(False, True)
    edit.startOperation()

    # Update the attribute values
    with arcpy.da.UpdateCursor(feature_class, field_name) as cursor:
        for row in cursor:
            old_value = row[0]
            if old_value in value_mapping:
                new_value = value_mapping[old_value]
                row[0] = new_value
                cursor.updateRow(row)

    # Stop the edit session and save changes
    edit.stopOperation()
    edit.stopEditing(True)

    print("Attribute values updated")

except Exception as e:
    print("Error updating attribute values: " + str(e))

Attribute values updated successfully.


Visual interpretation is now necessary to fill in the 'Ground Truth' field

In [2]:
# Calculate confusion matrix
arcpy.ia.ComputeConfusionMatrix(
    "AccuracyAssessmentPointsRF_10", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\AnalysisAndModelingFinalVisualInt.gdb\AccuracyRF"
)

<geoprocessing server result object object at 0x000001BB70EB0840>

# 60m resolution analysis


In [14]:
# Resample image
arcpy.management.Resample(
    "SanMarinoS2_all_bands", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\AnalysisAndModelingFinalVisualInt.gdb\SanMarinoS2_60m", 
    "60 60", 
    "NEAREST"
)

<Result 'C:\\Users\\Adian\\Documents\\ArcGIS\\Projects\\AnalysisAndModelingFinalVisualInt\\AnalysisAndModelingFinalVisualInt.gdb\\SanMarinoS2_60m'>

## SVM

In [15]:
# Train SVM and Generate an Esri classifier definition file (.ecd)
arcpy.ia.TrainSupportVectorMachineClassifier(
    "SanMarinoS2_60m", 
    "samples", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\SanMarinoTrainedSVM_60m.ecd", 
    '', 
    500, 
    "COLOR;MEAN;STD;COUNT;COMPACTNESS;RECTANGULARITY", 
    None
)

<geoprocessing server result object object at 0x000002553377B7B0>

In [16]:
# Classify the Sentinel image using the Esri classifier definition file (.ecd)
out_raster_dataset = arcpy.ia.ClassifyRaster(
    "SanMarinoS2_60m",
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\SanMarinoTrainedSVM_60m.ecd",
    ''
);
out_raster_dataset.save(
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\LandCoverSanMarinoSVM_60m.crf"
)

## RF

In [17]:
# Train RF and Classify raster
arcpy.stats.Forest(
    "PREDICT_RASTER", 
    "samples", 
    "Classcode",
    "CATEGORICAL",
    None, 
    None, 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2_60m #", 
    None,
    None, 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\rf60.crf",
    None,
    None, 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2_60m C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinal\AnalysisAndModelingFinal.gdb\SanMarinoS2_60m",
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\TrainedFeatures60.shp",
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\VariableImportance60.dbf", 
    "TRUE", 
    100,
    None,
    None,
    100,
    None, 
    10,
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\classificationPerformanceTable60.dbf", 
    None,
    "TRUE", 
    1,
    "FALSE"
)

<geoprocessing server result object object at 0x000002553377B7B0>

## Accuracy Assessment


In [19]:
# Duplicate existing SVM accuracy assessment points
arcpy.management.CopyFeatures(
    "AccuracyAssessmentPointsSVM",
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\AnalysisAndModelingFinalVisualInt.gdb\AccuracyAssessmentPointsSVM_60m", 
    '', 
    None, 
    None, 
    None
)

<Result 'C:\\Users\\Adian\\Documents\\ArcGIS\\Projects\\AnalysisAndModelingFinalVisualInt\\AnalysisAndModelingFinalVisualInt.gdb\\AccuracyAssessmentPointsSVM_60m'>

In [20]:
# Duplicate existing RF accuracy assessment points
arcpy.management.CopyFeatures(
    "AccuracyAssessmentPointsRF",
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\AnalysisAndModelingFinalVisualInt.gdb\AccuracyAssessmentPointsRF_60m", 
    '', 
    None, 
    None, 
    None
)

<Result 'C:\\Users\\Adian\\Documents\\ArcGIS\\Projects\\AnalysisAndModelingFinalVisualInt\\AnalysisAndModelingFinalVisualInt.gdb\\AccuracyAssessmentPointsRF_60m'>

### SVM

In [21]:
# Update classified column with the "LandCoverSanMarinoSVM_60m" values
arcpy.ia.UpdateAccuracyAssessmentPoints(
    "LandCoverSanMarinoSVM_60m", 
    "AccuracyAssessmentPointsSVM_60m", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\AnalysisAndModelingFinalVisualInt.gdb\AccuracyAssessmentPointsSVM_60m_updated", 
    "CLASSIFIED", 
    None, 
    None
)

<geoprocessing server result object object at 0x000002553377B660>

In [22]:
# Calculate confusion matrix
arcpy.ia.ComputeConfusionMatrix(
    "AccuracyAssessmentPointsSVM_60m_updated", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\AnalysisAndModelingFinalVisualInt.gdb\AccuracySVM_60m"
)

<geoprocessing server result object object at 0x000002553377B7B0>

### RF

In [1]:
# Update classified column with the RF 60m values
# 8 or 16 bpp raster must be used as input
arcpy.ia.UpdateAccuracyAssessmentPoints(
    "rf60_16bit.crf", 
    "AccuracyAssessmentPointsRF_60m", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\AnalysisAndModelingFinalVisualInt.gdb\AccuracyAssessmentPointsRF_60m_updated", 
    "CLASSIFIED", 
    None, 
    None
)

<geoprocessing server result object object at 0x000001FAB80B0540>

In [2]:
# Update ground truth identifiers with class values

# Path to the workspace and the feature class
workspace = r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt"

feature_class = "AccuracyAssessmentPointsRF_60m_updated"

# Field name to be updated
field_name = "Classified"


# Map values to match simple land cover
value_mapping = {
    0: 10,
    1: 20,
    2: 30
}

try:
    arcpy.env.workspace = workspace

    # Start an edit session
    edit = arcpy.da.Editor(workspace)
    edit.startEditing(False, True)
    edit.startOperation()

    # Update the attribute values
    with arcpy.da.UpdateCursor(feature_class, field_name) as cursor:
        for row in cursor:
            old_value = row[0]
            if old_value in value_mapping:
                new_value = value_mapping[old_value]
                row[0] = new_value
                cursor.updateRow(row)

    # Stop the edit session and save changes
    edit.stopOperation()
    edit.stopEditing(True)

    print("Attribute values updated")

except Exception as e:
    print("Error updating attribute values: " + str(e))

Attribute values updated successfully.


In [1]:
# Calculate confusion Matrix
arcpy.ia.ComputeConfusionMatrix(
    "AccuracyAssessmentPointsRF_60m_updated", 
    r"C:\Users\Adian\Documents\ArcGIS\Projects\AnalysisAndModelingFinalVisualInt\AnalysisAndModelingFinalVisualInt.gdb\AccuracyRF_60m"
)

<geoprocessing server result object object at 0x000001C5018551E0>